<a href="https://colab.research.google.com/github/hanbuck30/neural-layer/blob/main/RNN_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [255]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim

In [298]:
timesteps = 5 # 시점의 수. NLP에서는 보통 문장의 길이
input_size = 3 # 입력의 차원. NLP에서는 보통 단어 벡터의 차원
hidden_size = 8 # 은닉 상태의 크기. 메모리 셀의 용량
output_size=5
batch_size=1
inputs = torch.randn(batch_size,timesteps,input_size)
hidden_state_t=torch.zeros(hidden_size,requires_grad=True)


In [188]:
Wx = torch.randn(hidden_size, input_size,requires_grad=True) 
Wh = torch.randn(hidden_size, hidden_size,requires_grad=True)
b = torch.randn(hidden_size,requires_grad=True)
learning_rate = 0.1

In [46]:
print(Wx.shape)
print(Wh.shape)
print(b.shape)

torch.Size([8, 3])
torch.Size([8, 8])
torch.Size([8])


In [ ]:
inputs.shape

torch.Size([1, 5, 3])

In [245]:
Y=torch.LongTensor([[0,1,2,3,4]])
y_data=[Y]
Y.shape

torch.Size([1, 5])

In [191]:
b_fc=torch.randn(output_size,requires_grad=True)
w_fc=torch.randn(hidden_size, output_size,requires_grad=True)

In [302]:
class RNNModel(torch.nn.Module):
    
    def __init__(self,input_size,hidden_size,output_size,num_layers):
        super(RNNModel,self).__init__()
        self.input_size=input_size
        self.hidden_size=hidden_size
        self.num_layers=num_layers
        self.output_size = output_size
        self.Wx = torch.randn(hidden_size, input_size,requires_grad=True) 
        self.Wh = torch.randn(hidden_size, hidden_size,requires_grad=True)
        self.b = torch.randn(hidden_size,requires_grad=True)
        self.parameters=[]
        
        self.b_fc=torch.randn(output_size,requires_grad=True)
        self.w_fc=torch.randn(hidden_size, output_size,requires_grad=True)
        if num_layers >= 2:
            for k in range(1,num_layers):
                globals()['W{}'.format(k)]=torch.randn(hidden_size,hidden_size,requires_grad=True)
                globals()['b{}'.format(k)]=torch.randn(hidden_size,requires_grad=True)
                globals()['Wh{}'.format(k)]=torch.randn(hidden_size,hidden_size,requires_grad=True)
                globals()['hidden_state_t{}'.format(k)]=torch.zeros(hidden_size,requires_grad=True)
    
    def make_parameters(self,num_layers):
        global parameters
        parameters=[Wx,Wh,b,b_fc,w_fc]
        for k in range(1,num_layers):
                parameters.append(globals()['W{}'.format(k)])
                parameters.append(globals()['b{}'.format(k)])
                parameters.append(globals()['Wh{}'.format(k)])
                parameters.append(globals()['hidden_state_t{}'.format(k)]) 
           
    def rnn(self,inputs):
        hidden_state_t=torch.zeros(hidden_size,requires_grad=True)
        total_hidden_state=[]
        total_hidden_states=[]
        if self.num_layers ==1:
            for i in inputs:
                for input_t in i: 
                    output_t = torch.tanh(torch.matmul(Wx,input_t) + torch.matmul(Wh,hidden_state_t) + b)
                    total_hidden_states.append(output_t) 
                    hidden_state_t = output_t
                total_state = torch.stack(total_hidden_states,dim=0)
                total_hidden_state.append(total_state)
            total_state=torch.stack(total_hidden_state,dim=0)
            return total_state
        elif self.num_layers == 2: 
            for i in inputs:
                for input_t in i: 
                    output_t = torch.tanh(torch.matmul(Wx,input_t) + torch.matmul(Wh,hidden_state_t) + b)
                    hidden_state_t = output_t
                    output_t1 = torch.tanh(torch.matmul(W1,hidden_state_t) + torch.matmul(Wh1,hidden_state_t1) + b1)
                    hiddent_state_t1=output_t1
                    total_hidden_states.append(output_t1)
                total_state = torch.stack(total_hidden_states,dim=0)
                total_hidden_state.append(total_state)
            total_state=torch.stack(total_hidden_state,dim=0)
            return total_state
        elif self.num_layers > 2:
            for i in inputs:
                for input_t in i: 
                    output_t = torch.tanh(torch.matmul(Wx,input_t) + torch.matmul(Wh,hidden_state_t) + b)
                    hidden_state_t = output_t
                    output_t1 = torch.tanh(torch.matmul(W1,hidden_state_t) + torch.matmul(Wh1,hidden_state_t1) + b1)
                    hiddent_state_t1=output_t1
                    for m in range(2,num_layers):
                        globals()['output_t{}'.format(m)] = torch.tanh(torch.matmul(globals()['W{}'.format(m)],globals()['hidden_state_t{}'.format(m)]) + torch.matmul(globals()['Wh{}'.format(m)],globals()['hidden_state_t{}'.format(m-1)]) + globals()['b{}'.format(m)])
                        globals()['hidden_state_t{}'.format(m)] = globals()['output_t{}'.format(m)]
                    total_hidden_states.append(globals()['output_t{}'.format(num_layers)])
                total_state = torch.stack(total_hidden_states,dim=0)
                total_hidden_state.append(total_state)
            total_state=torch.stack(total_hidden_state,dim=0)
            return total_state
    def fc(self,total_state):
        linear=torch.matmul(total_state,w_fc)+b_fc
        return linear
    def forward(self,inputs):
        x=self.rnn(inputs)
        v=self.fc(x)
        return v
    


In [303]:
model=RNNModel(input_size, hidden_size, output_size, 4)
model.make_parameters(4)
print(len(parameters))
net=model(inputs)
net

17


tensor([[[-1.1897, -2.8213,  4.8691,  3.8571,  2.0977],
         [-1.1897, -2.8213,  4.8691,  3.8571,  2.0977],
         [-1.1897, -2.8213,  4.8691,  3.8571,  2.0977],
         [-1.1897, -2.8213,  4.8691,  3.8571,  2.0977],
         [-1.1897, -2.8213,  4.8691,  3.8571,  2.0977]]],
       grad_fn=<AddBackward0>)

In [280]:
def RNNmodel(inputs,hidden_state_t,num_layers):
    total_hidden_state=[]
    total_hidden_states=[]
    def RNN(inputs,hidden_state_t,num_layers):
        for i in inputs:
            for input_t in i: 
                output_t = torch.tanh(torch.matmul(Wx,input_t) + torch.matmul(Wh,hidden_state_t) + b)
                total_hidden_states.append(output_t) 
                hidden_state_t = output_t
            total_state = torch.stack(total_hidden_states,dim=0)
            total_hidden_state.append(total_state)
        total_state=torch.stack(total_hidden_state,dim=0)
        return total_state
    def RNN_two(inputs,hidden_state_t,num_layers):
        for k in range(1,num_layers):
            globals()['W{}'.format(k)]=torch.randn(hidden_size,hidden_size,requires_grad=True)
            globals()['b{}'.format(k)]=torch.randn(hidden_size,requires_grad=True)
            globals()['Wh{}'.format(k)]=torch.randn(hidden_size,hidden_size,requires_grad=True)
            globals()['hidden_state_t{}'.format(k)]=torch.zeros(hidden_size,requires_grad=True)
        for i in inputs:
            for input_t in i: 
                output_t = torch.tanh(torch.matmul(Wx,input_t) + torch.matmul(Wh,hidden_state_t) + b)
                hidden_state_t = output_t
                output_t1 = torch.tanh(torch.matmul(W1,hidden_state_t) + torch.matmul(Wh1,hidden_state_t1) + b1)
                hiddent_state_t1=output_t1
                total_hidden_states.append(output_t1)
            total_state = torch.stack(total_hidden_states,dim=0)
            total_hidden_state.append(total_state)
        total_state=torch.stack(total_hidden_state,dim=0)
        return total_state
    def RNN_multi(inputs,hidden_state_t,num_layers):
        for k in range(1,num_layers):
            globals()['W{}'.format(k)]=torch.randn(hidden_size,hidden_size,requires_grad=True)
            globals()['b{}'.format(k)]=torch.randn(hidden_size,requires_grad=True)
            globals()['Wh{}'.format(k)]=torch.randn(hidden_size,hidden_size,requires_grad=True)
            globals()['hidden_state_t{}'.format(k)]=torch.zeros(hidden_size,requires_grad=True)
        for i in inputs:
            for input_t in i: 
                output_t = torch.tanh(torch.matmul(Wx,input_t) + torch.matmul(Wh,hidden_state_t) + b)
                hidden_state_t = output_t
                output_t1 = torch.tanh(torch.matmul(W1,hidden_state_t) + torch.matmul(Wh1,hidden_state_t1) + b1)
                hiddent_state_t1=output_t1
                for m in range(2,num_layers):
                    globals()['output_t{}'.format(m)] = torch.tanh(torch.matmul(globals()['W{}'.format(m)],globals()['hidden_state_t{}'.format(m)]) + torch.matmul(globals()['Wh{}'.format(m)],globals()['hidden_state_t{}'.format(m-1)]) + globals()['b{}'.format(m)])
                    globals()['hidden_state_t{}'.format(m)] = globals()['output_t{}'.format(m)]
                total_hidden_states.append(globals()['output_t{}'.format(num_layers-1)])
            total_state = torch.stack(total_hidden_states,dim=0)
            total_hidden_state.append(total_state)
        total_state=torch.stack(total_hidden_state,dim=0)
        return total_state
    def fc(total_state):
        linear=torch.matmul(total_state,w_fc)+b_fc
        return linear
    if num_layers == 1:
        v=fc(RNN(inputs,hidden_state_t,num_layers))
        return v
    elif num_layers== 2 :
        v=fc(RNN_two(inputs,hidden_state_t,num_layers))
        return v
    else:
        v=fc(RNN_multi(inputs,hidden_state_t,num_layers))
        return v
RNNmodel(inputs,hidden_state_t,2).shape
#RNNmodel(inputs,hidden_state_t,2)

torch.Size([1, 5, 5])

In [296]:
parameters=[Wx,Wh,b,b_fc,w_fc]
def make_parameters(num_layers):
    for k in range(1,num_layers):
            parameters.append(globals()['W{}'.format(k)])
            parameters.append(globals()['b{}'.format(k)])
            parameters.append(globals()['Wh{}'.format(k)])
            parameters.append(globals()['hidden_state_t{}'.format(k)])
make_parameters(num_layers)


In [297]:
len(parameters)

9

In [243]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(parameters, learning_rate)

/usr/local/lib/python3.8/dist-packages/torch/optim/adam.py:137: UserWarning: optimizer contains a parameter group with duplicate parameters; in future, this will cause an error; see github.com/pytorch/pytorch/issues/40967 for more information
  super(Adam, self).__init__(params, defaults)


In [304]:
for i in range(100):
    total_hidden_state=[]
    total_hidden_states=[]
    optimizer.zero_grad()
    outputs=RNNmodel(inputs,hidden_state_t,num_layers)
    loss = criterion(outputs.view(-1,output_size), Y.view(-1)) # view를 하는 이유는 Batch 차원 제거를 위해
    loss.backward() 
    optimizer.step() 

    
    result = outputs.data.numpy().argmax(axis=2) 
    print(i, "loss: ", loss.item(), "prediction: ", result, "true Y: ", y_data)

0 loss:  2.672586441040039 prediction:  [[2 2 2 2 3]] true Y:  [tensor([[0, 1, 2, 3, 4]])]
1 loss:  3.308218002319336 prediction:  [[1 0 0 4 0]] true Y:  [tensor([[0, 1, 2, 3, 4]])]
2 loss:  4.615926265716553 prediction:  [[2 4 2 2 2]] true Y:  [tensor([[0, 1, 2, 3, 4]])]
3 loss:  3.349600315093994 prediction:  [[3 0 1 0 2]] true Y:  [tensor([[0, 1, 2, 3, 4]])]
4 loss:  3.8808753490448 prediction:  [[2 3 2 3 2]] true Y:  [tensor([[0, 1, 2, 3, 4]])]
5 loss:  3.707132339477539 prediction:  [[4 0 2 0 2]] true Y:  [tensor([[0, 1, 2, 3, 4]])]
6 loss:  3.9745421409606934 prediction:  [[2 2 3 2 2]] true Y:  [tensor([[0, 1, 2, 3, 4]])]
7 loss:  5.564478874206543 prediction:  [[1 2 1 1 0]] true Y:  [tensor([[0, 1, 2, 3, 4]])]
8 loss:  3.197936534881592 prediction:  [[0 1 1 3 1]] true Y:  [tensor([[0, 1, 2, 3, 4]])]
9 loss:  3.5745692253112793 prediction:  [[2 1 2 1 3]] true Y:  [tensor([[0, 1, 2, 3, 4]])]
10 loss:  2.982217311859131 prediction:  [[1 3 1 3 1]] true Y:  [tensor([[0, 1, 2, 3, 4]])